In [1]:
import numpy as np
import pandas as pd
import pickle as pkl
import math
from scipy.stats import ttest_ind

import getpass

from helm.common.authentication import Authentication
from helm.common.perspective_api_request import PerspectiveAPIRequest, PerspectiveAPIRequestResult
from helm.common.request import Request, RequestResult
from helm.common.tokenization_request import TokenizationRequest, TokenizationRequestResult
from helm.proxy.accounts import Account

from helm.proxy.services.remote_service import RemoteService

import gurobipy as gp

In [2]:
valid_llms = ['anthropic/claude-3-5-sonnet-20240620',
              'openai/gpt-3.5-turbo-0125',
              'openai/gpt-4o-2024-05-13',
              'meta/llama-3.1-70b-instruct-turbo',
             'google/gemini-1.5-pro-001',
             'openai/o1-preview-2024-09-12']

In [3]:
api_key = getpass.getpass(prompt="Enter a valid API key: ")
auth = Authentication(api_key='REDACTED')
service = RemoteService("REDACTED")

# Access account and show my current quotas and usages
account: Account = service.get_account(auth)
print(account.usages.keys())

for key in account.usages.keys():
    print(key, account.usages[key])

Enter a valid API key:  ········


dict_keys(['gpt3', 'gpt4', 'codex', 'jurassic', 'gooseai', 'cohere', 'dall_e', 'together_vision', 'simple', 'together', 'anthropic', 'google', 'o1'])
gpt3 {'daily': Usage(period='2025-1-19', used=155624, quota=None), 'monthly': Usage(period='2025-1', used=155624, quota=None), 'total': Usage(period='all', used=1535504, quota=10000000)}
gpt4 {'daily': Usage(period='2025-1-19', used=152343, quota=None), 'monthly': Usage(period='2025-1', used=152343, quota=None), 'total': Usage(period='all', used=1664151, quota=10000000)}
codex {'daily': Usage(period=None, used=0, quota=10000)}
jurassic {'daily': Usage(period=None, used=0, quota=10000)}
gooseai {'daily': Usage(period=None, used=0, quota=10000)}
cohere {'daily': Usage(period=None, used=0, quota=10000)}
dall_e {'daily': Usage(period=None, used=0, quota=5)}
together_vision {'daily': Usage(period=None, used=0, quota=30)}
simple {'daily': Usage(period=None, used=0, quota=10000)}
together {'daily': Usage(period='2025-1-19', used=155412, quota=No

In [4]:
use_same_ingredients = True #Use same ingredients as the original menu
emissions_constraint = 0.25 #At most 25% of the expected emissions of a choice from the original menu
animals_constraint = 1.0 #At most the expected animal usage of a choice from the original menu

ratings = True # Whether to rate each generated recipe by expected preferences of the target population
direct = False # No IQP; just directly generate a revised menu 

n_new = 36 # Number of new recipes to generate
if not direct:
    n_new = 20

sim_lambda = 100 # Tradeoff between sum of expected satisfaction for each recipe individually and diversity of recipes

menu_loc = 'original-nature-sust-menu_no_v.txt'

with open(menu_loc, 'r') as file:
    orig_menu = file.read()
    
llm = valid_llms[5]#'anthropic/claude-3-5-sonnet-20240620'#'gemini-1.5-pro'
assert llm in valid_llms

In [7]:
# Poore and Nemecek. If not in P&N, then the table used in the Banerjee et al. Nature Sustainability paper (Appendix). 
# Per kilogram
# https://faunalytics.org/animal-product-impact-scales/
# https://ourworldindata.org/grapher/kilograms-meat-per-animal 

ingredient_emissions_map = pkl.load(open('ingredient_emissions_map.pkl', 'rb'))
ingredient_animal_lives_map = pkl.load(open('ingredient_animal_lives_map.pkl', 'rb'))

In [8]:
orig_ingredients_lower = pkl.load(open('orig_ingredients.pkl', 'rb'))

In [2]:
orig_recipes_main_ingredient = pkl.load(open('orig_recipes_main_ingredient.pkl', 'rb'))

In [545]:
direct_msg = "You are a brilliant chef experienced at creating sustainable and delicious food. Here is a menu:\n" + orig_menu + "\
\n Please generate a revised menu, with the same number of recipes ({n}) and no new ingredients other than tofu, lentils, mushrooms, chickpeas, eggs, and cheese.\n\
 Design the menu to achieve at least a {frac}% CO2 emissions reduction in people's choices while maintaining or improving patron satisfaction\
 with their set of choices. Patrons will be American omnivores. Emissions will be computed based on the main (first) ingredient. \n\
 Please output each recipe in same format as this example:\n\
 Tofu curry ramen\n\
 Fried tofu, noodles, curry broth, pak choi, pickled onions.\n\
 Appealing description.\n\
 The ingredients must be in order of usage, i.e the main ingredient must come first.\n\
 Very important: you must only use ingredients in the original menu or the list above. For every ingredient, there must be an exact match in the original menu or the list above.\
 Do not worsen cost, nutrition, animal welfare (number of animals used, computed based on the first ingredient), or preparation time.\n\
 Do not include any stars, asterisks, hashtags, underscores. Do not number the recipes. Do not include any text other than recipe information, e.g. do not say `Here are the recipes'.".format(n=n_new,
                                                                                                                                                                                              frac=(1-emissions_constraint)*100)

iqp_msg = "You are a brilliant chef experienced at creating sustainable and delicious food. Here is a menu:\n" + orig_menu + "\
\n Please generate {n} new, delicious, and diverse vegan or vegetarian dishes from this set of ingredients. You are also allowed to use tofu, lentils, mushrooms, chickpeas, eggs, and cheese.\
 Patrons will be American omnivores. \
 Please output in same format as this example:\n\
 Tofu curry ramen\n\
 Fried tofu, noodles, curry broth, pak choi, pickled onions.\n\
 Appealing description.\n\
 The ingredients must be in order of usage, i.e the main ingredient must come first.\n\
 Very important: you must only use ingredients in the original menu or the list above. For every ingredient, there must be an exact match in the original menu or the list above.\
 Do not worsen CO2 emissions, cost, nutrition, or preparation time. Emissions will be computed based on the main (first) ingredient. \n\
 Do not include any stars, asterisks, hashtags, underscores. Do not number the recipes. Do not include any text other than recipe information, e.g. do not say `Here are the recipes'.".format(n=n_new)

msg = iqp_msg

if direct:
    msg = direct_msg
print(msg)

You are a brilliant chef experienced at creating sustainable and delicious food. Here is a menu:
Chicken Curry Ramen. Japanese fried chicken & noodles in a delicious curry broth. Topped with pak choi & pickled onions.

Pork Ribs. Pork Ribs smothered with Kentucky style BBQ sauce. 

Beef Bourguignon. 6oz Black Angus beef burger topped with crispy bacon, red wine braised onions, Raclette cheese.

Chicken Katsu Curry. Succulent chicken in a crispy Japanese panko crumb with mild curry sauce, pickles and steamed rice.

Pepperoni Melt. This one’s got our triple cheese blend, tomato sauce and pepperoni.

Lamb Meatballs with Spaghetti. Handmade lamb patties grilled and topped with Greek yoghurt, tomato sauce, and onions. 

Aromatic Duck Rolls. Aromatic roast duck, hoisin sauce, cucumber, spring onion, rolled in rice paper.

Slow Cooked Beef in Chianti Sauce. Beef shin and shallots slow cooked in rich Chianti red wine sauce with a cheesy dumpling.

Pork Porchetta. Slow-roasted pork belly in her

In [546]:
def compute_emissions(main_ingredient, emissions_map):
    for key in emissions_map:
        for val in emissions_map[key]:
            if val in main_ingredient:
                return key

def compute_animal_lives(main_ingredient, animal_map):
    for key in animal_map:
        for val in animal_map[key]:
            if val in main_ingredient:
                return key
    return 0

def check_ingredient(ingredient, ingredient_set):
    processed_ingredient = process_ingredient(ingredient)
    
    if processed_ingredient in ingredient_set:
        return True
    if any(processed_ingredient in base_ingredient for base_ingredient in ingredient_set):
        return True
    if any(base_ingredient in processed_ingredient for base_ingredient in ingredient_set):
        return True

def process_ingredient(ingredient):
    if ingredient[-1] == '.':
        ingredient = ingredient[:-1]
    return ingredient.lower().strip()

In [547]:
"""
Check:
- Requested number of recipes
- All ingredients must be mentioned in the original menu, exact match only
"""
def check_response(response, expected_num_recipes, check_main_ingredient_only=True):
    recipes = response.strip().split('\n\n')

    num_recipes = len(recipes)

    invalid_ingredients = []
    for recipe in recipes:
        parts = recipe.split('\n')
        ingredients = parts[1].split(', ')
        print('ingredients: ', ingredients)
        if check_main_ingredient_only:
            if not check_ingredient(ingredients[0], orig_ingredients_lower):
                invalid_ingredients.append(ingredients[0])
        else:
            for ingredient in ingredients:
                if not check_ingredient(ingredient, orig_ingredients_lower):
                    invalid_ingredients.append(ingredient)                
    
    return num_recipes, invalid_ingredients

In [548]:
def prompt_llm(prompt, llm):
    # Make a request
    mx_tokens = 4096
    if llm == 'openai/o1-preview-2024-09-12':
        mx_tokens = 32768
    request = Request(
        model=llm, prompt=prompt, echo_prompt=False,
        max_tokens=mx_tokens,
    )
    request_result: RequestResult = service.make_request(auth, request)
    return request_result.completions[0].text

def gen_ranking_str(all_recipes, idx_start, idx_end):
    ranking_str = ''
    idx = idx_start
    while idx < idx_end and idx < len(all_recipes):
        ranking_str += all_recipes[idx][0] + '\n'  
        idx+=1 
    return ranking_str


In [549]:
response = prompt_llm(msg, llm)

In [550]:
response

"Creamy Mushroom Tagliatelle  \nMushrooms, tagliatelle pasta, mascarpone cheese, baby spinach, garlic oil, basil.  \nAn indulgent pasta dish featuring sautéed mushrooms and baby spinach in a creamy mascarpone sauce, tossed with tagliatelle and seasoned with garlic oil and fresh basil.\n\nChickpea Curry with Rice  \nChickpeas, mild curry sauce, pickles, steamed rice.  \nA flavorful and hearty chickpea curry served with steamed rice and accompanied by tangy pickles.\n\nEggplant Parmesan  \nAubergine, tomato sauce, mozzarella, Grana Padano cheese, basil.  \nLayers of tender aubergine baked with rich tomato sauce, melted mozzarella, and Grana Padano cheese, garnished with fresh basil.\n\nVegetable and Tofu Stir-Fry  \nTofu, noodles, pak choi, red peppers, red onions, garlic & ginger sesame soy.  \nA vibrant stir-fry of crispy tofu and fresh vegetables tossed with noodles in a savory garlic and ginger sesame soy sauce.\n\nSpinach and Feta Stuffed Mushrooms  \nMushrooms, baby spinach, feta c

In [551]:
response.strip().split('\n\n')

['Creamy Mushroom Tagliatelle  \nMushrooms, tagliatelle pasta, mascarpone cheese, baby spinach, garlic oil, basil.  \nAn indulgent pasta dish featuring sautéed mushrooms and baby spinach in a creamy mascarpone sauce, tossed with tagliatelle and seasoned with garlic oil and fresh basil.',
 'Chickpea Curry with Rice  \nChickpeas, mild curry sauce, pickles, steamed rice.  \nA flavorful and hearty chickpea curry served with steamed rice and accompanied by tangy pickles.',
 'Eggplant Parmesan  \nAubergine, tomato sauce, mozzarella, Grana Padano cheese, basil.  \nLayers of tender aubergine baked with rich tomato sauce, melted mozzarella, and Grana Padano cheese, garnished with fresh basil.',
 'Vegetable and Tofu Stir-Fry  \nTofu, noodles, pak choi, red peppers, red onions, garlic & ginger sesame soy.  \nA vibrant stir-fry of crispy tofu and fresh vegetables tossed with noodles in a savory garlic and ginger sesame soy sauce.',
 'Spinach and Feta Stuffed Mushrooms  \nMushrooms, baby spinach, f

In [552]:
num_recipes, invalid_ingredients = check_response(response, n_new, check_main_ingredient_only=False)

ingredients:  ['Mushrooms', 'tagliatelle pasta', 'mascarpone cheese', 'baby spinach', 'garlic oil', 'basil.  ']
ingredients:  ['Chickpeas', 'mild curry sauce', 'pickles', 'steamed rice.  ']
ingredients:  ['Aubergine', 'tomato sauce', 'mozzarella', 'Grana Padano cheese', 'basil.  ']
ingredients:  ['Tofu', 'noodles', 'pak choi', 'red peppers', 'red onions', 'garlic & ginger sesame soy.  ']
ingredients:  ['Mushrooms', 'baby spinach', 'feta cheese', 'garlic oil.  ']
ingredients:  ['Lentils', 'chargrilled brioche bun', 'baby gem lettuce', 'plum tomato', 'Dijon mayonnaise.  ']
ingredients:  ['Mozzarella', 'tomato sauce', 'grilled aubergines', 'roasted peppers', 'oven-roasted tomatoes', 'fresh basil.  ']
ingredients:  ['Tofu in crispy Japanese panko crumb', 'mild curry sauce', 'pickles', 'steamed rice.  ']
ingredients:  ['Falafel', 'mixed leaf salad', 'cucumber', 'plum tomatoes', 'red onion', 'tahini dip.  ']
ingredients:  ["Goat's cheese", 'roasted beetroot', 'mixed leaf salad', 'pomegranate

In [553]:
invalid_ingredients, num_recipes

([], 21)

In [554]:
valid_response_generated = (num_recipes >= n_new) and len(invalid_ingredients) == 0
valid_response_generated, num_recipes

(True, 21)

In [555]:
max_corrections = 5
iters = 0
while not valid_response_generated and iters <= max_corrections:
    corrective_msg = ''
    if num_recipes != n_new:
        corrective_msg += 'You generated an invalid number of recipes ({act} rather than {expected}), as determined by line breaks.\
        This may also be due to not following the formatting instructions.\n'.format(act=num_recipes, expected=n_new)

    if len(invalid_ingredients) > 0:
        corrective_msg += 'You used invalid ingredients: {invld}. You may have been close, but recall that every ingredient must have an EXACT match in the original menu.\n'.format(invld=invalid_ingredients)

    corrective_msg += 'Now I\'ll repeat the original prompt:\n'

    new_msg = corrective_msg + msg

    print(new_msg)
    
    response = prompt_llm(new_msg,llm)
    num_recipes, invalid_ingredients = check_response(response, n_new)
    valid_response_generated = (num_recipes == n_new) and len(invalid_ingredients) == 0

    iters += 1

print(corrective_msg)

You generated an invalid number of recipes (1 rather than 20), as determined by line breaks.        This may also be due to not following the formatting instructions.
Now I'll repeat the original prompt:



In [556]:
if not valid_response_generated:
    assert 0

In [557]:
invalid_ingredients

[]

In [558]:
recipes = response.split('\n\n')

In [559]:
recipes

['Creamy Mushroom Tagliatelle  \nMushrooms, tagliatelle pasta, mascarpone cheese, baby spinach, garlic oil, basil.  \nAn indulgent pasta dish featuring sautéed mushrooms and baby spinach in a creamy mascarpone sauce, tossed with tagliatelle and seasoned with garlic oil and fresh basil.',
 'Chickpea Curry with Rice  \nChickpeas, mild curry sauce, pickles, steamed rice.  \nA flavorful and hearty chickpea curry served with steamed rice and accompanied by tangy pickles.',
 'Eggplant Parmesan  \nAubergine, tomato sauce, mozzarella, Grana Padano cheese, basil.  \nLayers of tender aubergine baked with rich tomato sauce, melted mozzarella, and Grana Padano cheese, garnished with fresh basil.',
 'Vegetable and Tofu Stir-Fry  \nTofu, noodles, pak choi, red peppers, red onions, garlic & ginger sesame soy.  \nA vibrant stir-fry of crispy tofu and fresh vegetables tossed with noodles in a savory garlic and ginger sesame soy sauce.',
 'Spinach and Feta Stuffed Mushrooms  \nMushrooms, baby spinach, f

In [560]:
all_recipes = []

# All_recipes tuple: name, description, emissions, animals, LLM_generated

# Add LLM generated recipes
for recipe in recipes:
    print(recipe)
    parts = recipe.split('\n')
    ingredients = parts[1].split(', ')
    name = parts[0]
    #print(ingredients[0])
    description = parts[2]

    emissions = compute_emissions(ingredients[0].lower(), ingredient_emissions_map)
    animals = compute_animal_lives(ingredients[0].lower(), ingredient_animal_lives_map)
    
    if emissions is None:
        print(name)
        print(ingredients[0].lower())
        assert 0
    
    all_recipes.append([name, description, emissions, animals, 1, ingredients])

Creamy Mushroom Tagliatelle  
Mushrooms, tagliatelle pasta, mascarpone cheese, baby spinach, garlic oil, basil.  
An indulgent pasta dish featuring sautéed mushrooms and baby spinach in a creamy mascarpone sauce, tossed with tagliatelle and seasoned with garlic oil and fresh basil.
Chickpea Curry with Rice  
Chickpeas, mild curry sauce, pickles, steamed rice.  
A flavorful and hearty chickpea curry served with steamed rice and accompanied by tangy pickles.
Eggplant Parmesan  
Aubergine, tomato sauce, mozzarella, Grana Padano cheese, basil.  
Layers of tender aubergine baked with rich tomato sauce, melted mozzarella, and Grana Padano cheese, garnished with fresh basil.
Vegetable and Tofu Stir-Fry  
Tofu, noodles, pak choi, red peppers, red onions, garlic & ginger sesame soy.  
A vibrant stir-fry of crispy tofu and fresh vegetables tossed with noodles in a savory garlic and ginger sesame soy sauce.
Spinach and Feta Stuffed Mushrooms  
Mushrooms, baby spinach, feta cheese, garlic oil.  
L

In [561]:
len(all_recipes)

21

In [562]:
invalid_ingredients

[]

In [563]:
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [564]:
# Edit emissions and animal constraints to expected values
# \sum e_i s_i <= kConstraint --> \sum e_i r_i s_i <= Constraint (\sum r_i s_i) where higher r_i is more liked
# Constraint should be \sum e_i w_i in original menu. Expected emissions in original menu. 
def solve_iqp(all_recipes, orig_subset, emissions_frac, animal_frac):
    # Create a new model
    m = gp.Model()
    
    # Create variables
    vars = []
    obj = 0
    k = len(orig_subset)
    
    card_constraint = 0
    emissions_constraint = 0
    animals_constraint = 0
    #nutr_constraint = 0
    #cost_constraint = 0

    rating_idx = 6
    emissions_idx = 2
    animals_idx = 3

    orig_subset_preference_totals = sum(r[rating_idx] for r in orig_subset)
    emissions_ev = sum(r[emissions_idx]*r[rating_idx] for r in orig_subset)/orig_subset_preference_totals
    animals_ev = sum(r[animals_idx]*r[rating_idx] for r in orig_subset)/orig_subset_preference_totals

    print('orig_subset_preference_totals: ', orig_subset_preference_totals)
    print('emissions_ev: ', emissions_ev)
    print('animals_ev: ', animals_ev)
    
    # Average across the original menu
    #emissions_val = np.mean([r[2] for r in orig_subset])
    #animals_val = np.mean([r[3] for r in orig_subset])
    
    for i in range(len(all_recipes)):
        print(i)
        this_var = m.addVar(vtype='B', name='recipe_' + str(i))
        rating = all_recipes[i][rating_idx]
        obj += rating*this_var
        vars.append(this_var)
        card_constraint += this_var
        emissions_constraint += this_var*all_recipes[i][emissions_idx]*all_recipes[i][rating_idx]
        animals_constraint += this_var*all_recipes[i][animals_idx]*all_recipes[i][rating_idx]
    
    similarity = 0
    for i in range(len(all_recipes)):
        for j in range(len(all_recipes)):
            #sim_ingr = similar(all_recipes[i]['ingredients'], all_recipes[j]['ingredients'])
            sim_name = similar(all_recipes[i][0], all_recipes[j][0])
            similarity = sim_name*vars[i]*vars[j]
            #similarity += (sim_ingr + sim_name)*vars[i]*vars[j]
    
            #print('names: ', recipe_data[i]['name'], recipe_data[j]['name'], sim_name)
            #print('ingr: ', recipe_data[i]['ingredients'], recipe_data[j]['ingredients'], sim_ingr)
    
    obj -= sim_lambda*similarity
    
    #obj = 0
    
    # Set objective function
    m.setObjective(obj, gp.GRB.MAXIMIZE)
    
    # Add constraints
    m.addConstr(card_constraint == k)

    sumRiSi = sum(all_recipes[i][rating_idx]*vars[i] for i in range(len(vars)))
    m.addConstr(emissions_constraint <= emissions_frac*emissions_ev*sumRiSi)
    m.addConstr(animals_constraint <= animal_frac*animals_ev*sumRiSi)
    
    # Solve it!
    m.optimize()

    if not hasattr(m, 'objVal'):
        return None
    
    print(f"Optimal objective value: {m.objVal}")
    selected_recipes = []
    
    idx = 0
    for var in vars:
        print(var.X)
        if var.X == 1.0:
            selected_recipes.append(idx)
        idx += 1   

    return selected_recipes

def select_recipe_subset(all_recipes, orig_subset, emissions_frac, animal_frac):
    # Set up IQP
    selected_recipes = solve_iqp(all_recipes, orig_subset, emissions_frac, animal_frac)

    tries = 0
    while selected_recipes is None and tries < 5:
        emissions_frac += 0.1
        print('Now rerunning with emissions_frac:', emissions_frac)
        selected_recipes = solve_iqp(all_recipes, orig_subset, emissions_frac, animal_frac)        
        tries += 1
    
    return selected_recipes


In [565]:
# End now if direct
orig_recipes = orig_menu.split('\n\n')

orig_subset = []
# Add original recipes to all_recipes
for recipe in orig_recipes:
    name = recipe.split('.')[0]
    description = recipe.split('.')[1]
    print(name)
    main_ingredient = orig_recipes_main_ingredient[name]

    emissions = compute_emissions(main_ingredient.lower(), ingredient_emissions_map)
    animals = compute_animal_lives(main_ingredient.lower(), ingredient_animal_lives_map)
    
    orig_subset.append([name, description, emissions, animals, 0, None])

len(orig_subset)

Chicken Curry Ramen
Pork Ribs
Beef Bourguignon
Chicken Katsu Curry
Pepperoni Melt
Lamb Meatballs with Spaghetti
Aromatic Duck Rolls
Slow Cooked Beef in Chianti Sauce
Pork Porchetta
Cured Salmon Sashimi
Beef Brisket and Venison Tagliatelle
Rustica Chorizo Pizza
Butterfly Chicken Burger
American Hot Pizza
Beef Angus Burger
Beer Battered Fish with Chips
Wild Boar Polpette
Steak
Canelloni
Supreme Pizza
Goat's Cheese Calzone
Vegan Spaghetti Lentil Ragu
Jack Fruit Arrabiata
Halloumi Sticks with Mayo Avocado Dip
Cheese Salad
Falafel with Tahini
Four Cheese Margherita
Vegan Meatballs
Bufala Caprese
Butternut Squash Salad
Panchetta Carbonara
Lentil Linguine Ragu
Aegean Slaw
Goat's Cheese Salad
Harusami Aubergine
Inari Taco


36

In [566]:

if direct:
    selected_subset = all_recipes
else:    
    all_recipes = all_recipes + orig_subset

    print(len(all_recipes))
    
    if ratings: 
        rankings = []
        start_idx = 0
        window_size = 5
        end_idx = window_size
        
        while start_idx < len(all_recipes):
            ranking_str = gen_ranking_str(all_recipes, start_idx, end_idx)
            ranking_msg = 'Here are {nr} recipes. Please rate them on a scale of 1-10 based on standard American omnivore taste preferences, 1 being unappealing and 10 being appealing. Output only a comma-separated list of {nr} numbers, from 1 to 10.\n'.format(nr=len(all_recipes[start_idx:end_idx])) + ranking_str
            print(ranking_msg)
            response_ranking = prompt_llm(ranking_msg, llm)
            print(response_ranking)
            this_rankings = [int(r) for r in response_ranking.split(',')]

            rankings += this_rankings
            start_idx += window_size
            end_idx += window_size
    
        if len(rankings) != len(all_recipes):
            print(len(rankings))
            assert 0
    else:
        rankings = [10]*len(all_recipes)
    
    for r in range(len(all_recipes)):
        all_recipes[r].append(rankings[r])

    orig_subset = all_recipes[-len(orig_recipes):]

57


In [567]:
len(all_recipes)

57

In [568]:
animals_constraint

1.0

In [569]:
if not direct:
    selected_recipes = select_recipe_subset(all_recipes, orig_subset, emissions_constraint, animals_constraint)

    metrics_selected = {'Animals': 0, 'Emissions': 0}
    total_emissions = 0
    total_animals = 0
    for idx in selected_recipes:
        print(all_recipes[idx][0])
    
        total_emissions += all_recipes[idx][2]
        total_animals += all_recipes[idx][3]
    
    emissions_val = np.mean([r[2] for r in orig_subset])
    animals_val = np.mean([r[3] for r in orig_subset])
    avg_emissions = total_emissions / len(selected_recipes)
    avg_animals = total_animals / len(selected_recipes)
    
    print('Orig avg emissions :', emissions_val)
    print('Orig avg animals :', animals_val)
    print('Avg emissions in selected subset: ', avg_emissions)
    print('Avg animals in selected subset: ', avg_animals)

    selected_subset = [all_recipes[s] for s in selected_recipes]

orig_subset_preference_totals:  360
emissions_ev:  27.302777777777777
animals_ev:  0.06347226829792113
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (mac64[x86] - Darwin 24.2.0 24C101)

CPU model: Intel(R) Core(TM) i5-8257U CPU @ 1.40GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 3 rows, 57 columns and 171 nonzeros
Model fingerprint: 0x1432fa37
Model has 1 quadratic objective term
Variable types: 0 continuous, 57 integer (57 binary)
Coefficient statistics:
  Matrix range     [5e-01, 9e+02]
  Objective range  [1e+01, 1e+01]
  QObjective range [2e+02, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [4e+01, 4e+01]
Found heuristic solution: objective 260.0000000
Presolve removed 1 rows and 47 columns
Presolve time: 0.01s
Presolved: 2 rows, 10 columns, 19 nonzero

In [570]:
all_recipes

[['Creamy Mushroom Tagliatelle  ',
  'An indulgent pasta dish featuring sautéed mushrooms and baby spinach in a creamy mascarpone sauce, tossed with tagliatelle and seasoned with garlic oil and fresh basil.',
  0.5,
  0,
  1,
  ['Mushrooms',
   'tagliatelle pasta',
   'mascarpone cheese',
   'baby spinach',
   'garlic oil',
   'basil.  '],
  10],
 ['Chickpea Curry with Rice  ',
  'A flavorful and hearty chickpea curry served with steamed rice and accompanied by tangy pickles.',
  1.8,
  0,
  1,
  ['Chickpeas', 'mild curry sauce', 'pickles', 'steamed rice.  '],
  10],
 ['Eggplant Parmesan  ',
  'Layers of tender aubergine baked with rich tomato sauce, melted mozzarella, and Grana Padano cheese, garnished with fresh basil.',
  0.5,
  0,
  1,
  ['Aubergine',
   'tomato sauce',
   'mozzarella',
   'Grana Padano cheese',
   'basil.  '],
  10],
 ['Vegetable and Tofu Stir-Fry  ',
  'A vibrant stir-fry of crispy tofu and fresh vegetables tossed with noodles in a savory garlic and ginger sesame

In [571]:
len(orig_subset)

36

In [572]:
orig_subset

[['Chicken Curry Ramen',
  ' Japanese fried chicken & noodles in a delicious curry broth',
  9.9,
  0.5882352941176471,
  0,
  None,
  10],
 ['Pork Ribs',
  ' Pork Ribs smothered with Kentucky style BBQ sauce',
  12.3,
  0.01040582726326743,
  0,
  None,
  10],
 ['Beef Bourguignon',
  ' 6oz Black Angus beef burger topped with crispy bacon, red wine braised onions, Raclette cheese',
  99.5,
  0.0027442371020856204,
  0,
  None,
  10],
 ['Chicken Katsu Curry',
  ' Succulent chicken in a crispy Japanese panko crumb with mild curry sauce, pickles and steamed rice',
  9.9,
  0.5882352941176471,
  0,
  None,
  10],
 ['Pepperoni Melt',
  ' This one’s got our triple cheese blend, tomato sauce and pepperoni',
  55.9,
  0,
  0,
  None,
  10],
 ['Lamb Meatballs with Spaghetti',
  ' Handmade lamb patties grilled and topped with Greek yoghurt, tomato sauce, and onions',
  39.7,
  0,
  0,
  None,
  10],
 ['Aromatic Duck Rolls',
  ' Aromatic roast duck, hoisin sauce, cucumber, spring onion, rolled in

In [574]:
orig_df = pd.DataFrame(orig_subset)#pd.DataFrame(all_recipes[19:])
if direct:
    col_map = {0:'Name', 1:'Description', 2:'Emissions', 3:'Animal Lives', 4: 'LLM Generated', 5: 'Ingredients'}
else:
    col_map = {0:'Name', 1:'Description', 2:'Emissions', 3:'Animal Lives', 4: 'LLM Generated', 5: 'Ingredients', 6: 'Predicted Rating'}
orig_df = orig_df.rename(columns=col_map)
orig_df.to_csv('orig-menu-df.csv',index=False)

selected_df = pd.DataFrame(selected_subset)
selected_df = selected_df.rename(columns=col_map)
#selected_df = selected_df.sort_values(['LLM Generated', 'Predicted Rating'], ascending=[False, True]) #'Emissions', 
if not direct:
    selected_df = selected_df.sort_values(['LLM Generated', 'Predicted Rating'], ascending=[False, False]) #'Emissions', 
    #selected_df = selected_df.sort_values(['Predicted Rating', 'Emissions'], ascending=[True, True]) #'Emissions', 
if direct:
    selected_df.to_csv('direct-' + llm.split('/')[1] + '-generated-menu.csv',index=False)
else:
    selected_df.to_csv('iqp-' + llm.split('/')[1] + '-generated-menu.csv',index=False)    

In [575]:
orig_df['Animal Lives'].mean(), orig_df['Emissions'].mean()

(0.06347226829792113, 27.30277777777778)

In [576]:
selected_df['Animal Lives'].mean(), selected_df['Emissions'].mean()

(0.0630911242559648, 4.841666666666667)

In [577]:
# Text for original menu
orig_text = ''
for idx in range(orig_df.shape[0]):
    output = orig_df.iloc[idx, 0] + '. ' + orig_df.iloc[idx, 1]
    if output[-1] != '.':
        output += '.'
    orig_text += str(idx + 1) + '. ' + output + '\n'
print(orig_text)

1. Chicken Curry Ramen.  Japanese fried chicken & noodles in a delicious curry broth.
2. Pork Ribs.  Pork Ribs smothered with Kentucky style BBQ sauce.
3. Beef Bourguignon.  6oz Black Angus beef burger topped with crispy bacon, red wine braised onions, Raclette cheese.
4. Chicken Katsu Curry.  Succulent chicken in a crispy Japanese panko crumb with mild curry sauce, pickles and steamed rice.
5. Pepperoni Melt.  This one’s got our triple cheese blend, tomato sauce and pepperoni.
6. Lamb Meatballs with Spaghetti.  Handmade lamb patties grilled and topped with Greek yoghurt, tomato sauce, and onions.
7. Aromatic Duck Rolls.  Aromatic roast duck, hoisin sauce, cucumber, spring onion, rolled in rice paper.
8. Slow Cooked Beef in Chianti Sauce.  Beef shin and shallots slow cooked in rich Chianti red wine sauce with a cheesy dumpling.
9. Pork Porchetta.  Slow-roasted pork belly in herbed red wine sauce, served with roasted new potatoes and broccoli.
10. Cured Salmon Sashimi.  Dazzling beetroo

In [578]:
# Text for LLM generated menu
llm_text = ''
for idx in range(selected_df.shape[0]):
    output = selected_df.iloc[idx, 0] + '. ' + selected_df.iloc[idx, 1]
    if output[-1] != '.':
        output += '.'
    llm_text += str(idx + 1) + '. ' + output + '\n'
print(llm_text)

1. Creamy Mushroom Tagliatelle  . An indulgent pasta dish featuring sautéed mushrooms and baby spinach in a creamy mascarpone sauce, tossed with tagliatelle and seasoned with garlic oil and fresh basil.
2. Chickpea Curry with Rice  . A flavorful and hearty chickpea curry served with steamed rice and accompanied by tangy pickles.
3. Eggplant Parmesan  . Layers of tender aubergine baked with rich tomato sauce, melted mozzarella, and Grana Padano cheese, garnished with fresh basil.
4. Vegetable and Tofu Stir-Fry  . A vibrant stir-fry of crispy tofu and fresh vegetables tossed with noodles in a savory garlic and ginger sesame soy sauce.
5. Spinach and Feta Stuffed Mushrooms  . Large mushrooms stuffed with sautéed baby spinach and creamy feta cheese, drizzled with garlic oil and baked to perfection.
6. Lentil Veggie Burger  . A hearty lentil-based veggie burger served on a toasted brioche bun with fresh lettuce, tomato, and tangy Dijon mayonnaise.
7. Tofu Katsu Curry  . Succulent tofu coate

In [579]:
orig_menu_loc = 'orig-menu.txt'
with open(orig_menu_loc, 'w') as f:
    f.write(orig_text)

llm_menu_loc = 'iqp-' + llm.split('/')[1] + '-generated-menu.txt'
if direct:
    llm_menu_loc = 'direct-' + llm.split('/')[1] + '-generated-menu.txt'

with open(llm_menu_loc, 'w') as f:
    f.write(llm_text)